In [18]:
import pandas as pd
review = pd.read_json('Data/review_city.json', lines=True)
business = pd.read_json('Data/business_city.json', lines=True)
df_raw = pd.merge(review, business, how='left', on='business_id')
df_raw.head()

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,name,...,state,postal_code,latitude,longitude,stars_y,review_count,is_open,attributes,categories,hours
0,LG2ZaYiOgpr2DK_90pYjNw,V34qejxNsCbcgD8C0HVk-Q,HQl28KMwrEKHqhFrrDqVNQ,5,1,0,0,I love Deagan's. I do. I really do. The atmosp...,2015-12-05 03:18:11,Deagan's Kitchen & Bar,...,OH,44107,41.485192,-81.800145,4.0,417,1,"{'BusinessAcceptsCreditCards': 'True', 'Outdoo...","Restaurants, Gastropubs, Nightlife, American (...","{'Tuesday': '11:0-22:0', 'Wednesday': '11:0-22..."
1,qWHp2l2lysENZObh6MAulA,5lb0POg2t-AkMFx6603cYQ,ZlCSsWS07JulSBIQlrbF5w,5,1,0,0,Called for a 5:15 morning reservation. The dri...,2015-05-26 10:36:47,Black Cab & Limo,...,IL,,40.116420,-88.243383,4.5,28,1,{'WiFi': 'u'free''},"Airport Shuttles, Transportation, Taxis, Hotel...","{'Monday': '8:0-20:0', 'Tuesday': '8:0-20:0', ..."
2,Ne_2CSfcKIqXHmv_KC0uqg,6sJN_HlM_uwpfLJ1puf1Fg,oVuZtlCFg_zF090Nhtp0Ow,4,3,0,1,A close friend was in town and so instead of t...,2018-02-01 19:15:00,Choolaah Indian BBQ,...,PA,15206,40.460266,-79.923461,4.0,302,1,"{'RestaurantsTableService': 'False', 'Restaura...","Restaurants, Indian, Vegetarian, Vegan, Gluten...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-17:0', ..."
3,2mLlQ9n5Ln_fykOuX_b0-A,EZ6g9a17UdMjtR-wrZebmA,wSAGILK2CFbq5BVZ5D67-A,3,0,0,0,The service is attentive. The decor attractive...,2015-06-21 00:07:59,Adega,...,OH,44115,41.499615,-81.685922,4.0,201,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Mediterranean, Hotels, Beer, Wine & Spirits, B...","{'Monday': '6:30-22:0', 'Tuesday': '6:30-22:0'..."
4,Ymb8ZYPg2e8dvqhdj3Gkqw,hHFSpKcnDtJOs6M-JKFZ8Q,oCvxJBOs-Mbg-4oanPO7lQ,4,0,0,0,The ingredients are always fresh and I like th...,2016-10-26 14:53:42,Pita Pit,...,PA,15203,40.426647,-79.965902,3.5,57,1,"{'RestaurantsPriceRange2': '1', 'RestaurantsAt...","Fast Food, Greek, Breakfast & Brunch, Restaura...","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'..."


In [19]:
df_raw.columns
df = df_raw.drop(columns=['business_id', 'review_id', 'user_id', 'funny', 'cool', 'date', 'latitude', 'longitude', 'is_open', 'review_count'])
import numpy as np
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
import seaborn as sns
import re, string
import sys
import time
import collections

In [20]:
df.head()

,stars_x,useful,text,name,address,city,state,postal_code,stars_y,attributes,categories,hours
0,5,1,I love Deagan's. I do. I really do. The atmosp...,Deagan's Kitchen & Bar,14810 Detroit Ave,Lakewood,OH,44107,4.0,"{'BusinessAcceptsCreditCards': 'True', 'Outdoo...","Restaurants, Gastropubs, Nightlife, American (...","{'Tuesday': '11:0-22:0', 'Wednesday': '11:0-22..."
1,5,1,Called for a 5:15 morning reservation. The dri...,Black Cab & Limo,,Champaign,IL,,4.5,{'WiFi': 'u'free''},"Airport Shuttles, Transportation, Taxis, Hotel...","{'Monday': '8:0-20:0', 'Tuesday': '8:0-20:0', ..."
2,4,3,A close friend was in town and so instead of t...,Choolaah Indian BBQ,6114 Centre Ave,Pittsburgh,PA,15206,4.0,"{'RestaurantsTableService': 'False', 'Restaura...","Restaurants, Indian, Vegetarian, Vegan, Gluten...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-17:0', ..."
3,3,0,The service is attentive. The decor attractive...,Adega,2017 E 9th St,Cleveland,OH,44115,4.0,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Mediterranean, Hotels, Beer, Wine & Spirits, B...","{'Monday': '6:30-22:0', 'Tuesday': '6:30-22:0'..."
4,4,0,The ingredients are always fresh and I like th...,Pita Pit,2763 East Carson St,Pittsburgh,PA,15203,3.5,"{'RestaurantsPriceRange2': '1', 'RestaurantsAt...","Fast Food, Greek, Breakfast & Brunch, Restaura...","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'..."


In [21]:
brunch=df[df.categories.str.contains('Brunch')]

ValueError: cannot mask with array containing NA / NaN values

In [22]:
df.categories.str.contains('Brunch')

0         False
1         False
2         False
3          True
4          True
          ...  
942022    False
942023    False
942024    False
942025    False
942026    False
Name: categories, Length: 942027, dtype: object

In [23]:
df=df.dropna(axis=0, subset=['categories'])

In [24]:
brunch=df[df.categories.str.contains('Brunch')]

In [25]:
brunch.head()

,stars_x,useful,text,name,address,city,state,postal_code,stars_y,attributes,categories,hours
3,3,0,The service is attentive. The decor attractive...,Adega,2017 E 9th St,Cleveland,OH,44115,4.0,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Mediterranean, Hotels, Beer, Wine & Spirits, B...","{'Monday': '6:30-22:0', 'Tuesday': '6:30-22:0'..."
4,4,0,The ingredients are always fresh and I like th...,Pita Pit,2763 East Carson St,Pittsburgh,PA,15203,3.5,"{'RestaurantsPriceRange2': '1', 'RestaurantsAt...","Fast Food, Greek, Breakfast & Brunch, Restaura...","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'..."
27,4,0,Surprised they didn't open earlier in a Sunday...,DeLuca's Diner,2015 Penn Ave,Pittsburgh,PA,15222,4.0,"{'HasTV': 'True', 'RestaurantsPriceRange2': '1...","Diners, Breakfast & Brunch, American (Traditio...","{'Monday': '0:0-0:0', 'Tuesday': '6:30-14:0', ..."
42,3,0,The restaurant has a great atmosphere and the ...,One Red Door,49 Village Way,Hudson,OH,44236,3.5,"{'WiFi': 'u'no'', 'OutdoorSeating': 'True', 'B...","Cocktail Bars, Breakfast & Brunch, American (N...","{'Monday': '11:30-22:0', 'Tuesday': '11:30-22:..."
45,4,1,Super cute place! It's quite spacious with an ...,Ineffable Ca Phe,3920 Penn Ave,Pittsburgh,PA,15224,4.5,"{'RestaurantsTableService': 'False', 'BikePark...","Coffee & Tea, Restaurants, Food, Vietnamese, B...","{'Monday': '8:0-18:0', 'Tuesday': '8:0-18:0', ..."


In [66]:
steak=df[df.categories.str.contains('Steak')]
cajun=df[df.categories.str.contains('Cajun')]

In [60]:
steak.head()

,stars_x,useful,text,name,address,city,state,postal_code,stars_y,attributes,categories,hours
0,1,1,I gave this place 2 chances. i love japanese f...,Katana,3229 W Liberty Ave,Pittsburgh,PA,15216,3.0,"{'RestaurantsAttire': 'u'casual'', 'Restaurant...","Restaurants, Japanese, Sushi Bars, Chinese, St...","{'Monday': '16:30-22:0', 'Tuesday': '16:30-22:..."
1,4,0,Very fresh food and authentic Italian taste. T...,Grandview Saloon,1212 Grandview Ave,Pittsburgh,PA,15211,3.0,"{'Alcohol': 'u'full_bar'', 'WiFi': ''free'', '...","Restaurants, Steakhouses, Seafood, American (T...","{'Monday': '11:30-21:0', 'Tuesday': '11:30-21:..."
2,5,0,This is my favorite happy hour place in all of...,Tempest Oyster Bar,120 E Wilson St,Madison,WI,53703,4.0,"{'RestaurantsTakeOut': 'False', 'WiFi': 'u'no'...","Steakhouses, Restaurants, Bars, Seafood, Night...","{'Monday': '16:0-22:0', 'Tuesday': '16:0-22:0'..."
3,4,1,Ruth Chris Steakhouse is by far one of the bes...,Ruths Chris Steak House,6 PPG Pl,Pittsburgh,PA,15222,4.0,"{'BusinessParking': '{'garage': False, 'street...","Restaurants, Steakhouses","{'Monday': '17:0-22:0', 'Tuesday': '17:0-22:0'..."
4,3,5,"When it comes to Tomo, my biggest suggestions ...",Tomo Sushi & Hibachi,1293 W 9th St,Cleveland,OH,44113,3.0,"{'RestaurantsGoodForGroups': 'True', 'Business...","Sushi Bars, Restaurants, Japanese, Steakhouses","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."


In [28]:
brunch.shape

(88617, 12)

In [68]:
steak.shape

(32184, 12)

In [30]:
brunch.reset_index(drop=True, inplace=True)
steak.reset_index(drop=True, inplace=True)
cajun.reset_index(drop=True, inplace=True)

In [31]:
textbr=list(brunch.text)
textst=list(steak.text)
textca=list(cajun.text)

In [32]:
textbr = [ a.lower() for a in textbr ]
textst = [ a.lower() for a in textst ]
textca = [ a.lower() for a in textca ]

In [35]:
import string
textbr = ["".join([char for char in a if char not in string.punctuation]) for a in textbr]
textst = ["".join([char for char in a if char not in string.punctuation]) for a in textst]
textca = ["".join([char for char in a if char not in string.punctuation]) for a in textca]

In [36]:
from nltk import word_tokenize

In [39]:
wordsbr=[nltk.word_tokenize(a) for a in textbr]
wordsst=[nltk.word_tokenize(a) for a in textst]
wordsca=[nltk.word_tokenize(a) for a in textca]

In [41]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
filtered_wordsbr = [[word for word in a if word not in stop_words] for a in wordsbr]
filtered_wordsst = [[word for word in a if word not in stop_words] for a in wordsst]
filtered_wordsca = [[word for word in a if word not in stop_words] for a in wordsca]

In [42]:
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()

In [44]:
lemmatized_wordsbr = [[lmtzr.lemmatize(x) for x in a] for a in filtered_wordsbr]
lemmatized_wordsst = [[lmtzr.lemmatize(x) for x in a] for a in filtered_wordsst]
lemmatized_wordsca = [[lmtzr.lemmatize(x) for x in a] for a in filtered_wordsca]

In [46]:
lemmatized_wordsbr[1][:10]

['ingredient',
 'always',
 'fresh',
 'like',
 'customize',
 'everything',
 'price',
 'pitasalad',
 'reasonable',
 'delivery']

In [48]:
import itertools
word_listbr = list(itertools.chain.from_iterable(lemmatized_wordsbr))
word_listst = list(itertools.chain.from_iterable(lemmatized_wordsst))
word_listca = list(itertools.chain.from_iterable(lemmatized_wordsca))

In [49]:
from collections import Counter
freqbr = pd.DataFrame.from_dict(Counter(word_listbr), orient='index')

In [51]:
freqbr.sort_values(by=0, ascending=False)

,0
food,68156
good,54679
place,51875
great,42544
time,35893
...,...
concertgoer,1
tacto,1
retrolooking,1
auspicious,1


In [54]:
dictionarybr = list(freqbr[freqbr[0] > 15].index)
dictionarybr[:10]

['service',
 'attentive',
 'decor',
 'attractive',
 'menu',
 'far',
 'doused',
 'polenta',
 'paprika',
 'fingerling']

In [55]:
import numpy as np
word_embeddingbr = np.zeros([len(lemmatized_wordsbr), len(dictionarybr)])

In [57]:
for i,x in enumerate(lemmatized_wordsbr):
    for j,y in enumerate(dictionarybr):
        word_embeddingbr[i,j] = sum([z == y for z in x])
        

KeyboardInterrupt: 

In [63]:
print(brunch.categories[0:10])

0    Mediterranean, Hotels, Beer, Wine & Spirits, B...
1    Fast Food, Greek, Breakfast & Brunch, Restaura...
2    Diners, Breakfast & Brunch, American (Traditio...
3    Cocktail Bars, Breakfast & Brunch, American (N...
4    Coffee & Tea, Restaurants, Food, Vietnamese, B...
5    Diners, Bars, Nightlife, Greek, Cocktail Bars,...
6    Sandwiches, Vegetarian, Coffee & Tea, Breakfas...
7    Diners, Bars, Nightlife, Greek, Cocktail Bars,...
8    Ice Cream & Frozen Yogurt, Breakfast & Brunch,...
9    Bars, Breakfast & Brunch, Nightlife, Southern,...
Name: categories, dtype: object


In [69]:
print(steak.hours)

169       {'Monday': '16:30-22:0', 'Tuesday': '16:30-22:...
188       {'Monday': '11:30-21:0', 'Tuesday': '11:30-21:...
219       {'Monday': '16:0-22:0', 'Tuesday': '16:0-22:0'...
262       {'Monday': '17:0-22:0', 'Tuesday': '17:0-22:0'...
269       {'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...
                                ...                        
941785    {'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ...
941803    {'Monday': '4:0-20:45', 'Tuesday': '4:0-20:45'...
941806    {'Monday': '16:45-22:0', 'Tuesday': '16:0-21:0...
941894    {'Monday': '0:0-0:0', 'Tuesday': '16:0-21:0', ...
941988    {'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...
Name: hours, Length: 32184, dtype: object


In [70]:
for a in steak.hours:
    if 'Monday' in a:
        print(a)

{'Monday': '16:30-22:0', 'Tuesday': '16:30-22:0', 'Thursday': '16:30-22:0', 'Friday': '16:30-23:0', 'Saturday': '16:30-23:0', 'Sunday': '12:0-21:30'}
{'Monday': '11:30-21:0', 'Tuesday': '11:30-21:0', 'Wednesday': '11:30-21:0', 'Thursday': '11:30-21:0', 'Friday': '11:30-22:0', 'Saturday': '11:30-22:0', 'Sunday': '11:30-21:0'}
{'Monday': '16:0-22:0', 'Tuesday': '16:0-22:0', 'Wednesday': '16:0-22:0', 'Thursday': '16:0-22:0', 'Friday': '16:0-0:0', 'Saturday': '16:0-0:0', 'Sunday': '16:0-22:0'}
{'Monday': '17:0-22:0', 'Tuesday': '17:0-22:0', 'Wednesday': '17:0-22:0', 'Thursday': '17:0-22:0', 'Friday': '17:0-23:0', 'Saturday': '17:0-23:0', 'Sunday': '17:0-21:0'}
{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0', 'Wednesday': '11:0-23:0', 'Thursday': '11:0-23:0', 'Friday': '11:0-2:30', 'Saturday': '11:0-2:30', 'Sunday': '11:0-23:0'}
{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0', 'Wednesday': '11:0-22:0', 'Thursday': '11:0-22:0', 'Friday': '11:0-23:0', 'Saturday': '11:0-23:0', 'Sunday': '10:0-22

TypeError: argument of type 'NoneType' is not iterable